In [ ]:
import numpy as np
import starry
import astropy.units as u
from astropy.time import Time
from astropy.table import vstack
import sys
sys.path.append("../volcano")
from utils import get_body_ephemeris

np.random.seed(42)

starry.config.lazy = False

In [ ]:
%matplotlib inline
%run notebook_setup.py

In [ ]:
# Pick some random times in a 20yr period and get 2day ephemeris for each
# of those dates
start = Time("2000-01-01", format="isot")
stop = Time("2020-01-01", format="isot")
start_times = np.random.uniform(start.mjd, stop.mjd, 50)

eph_list_io = []
eph_list_jup = []

# This takes a while
for t in start_times:
    delta_t = 2  # interval
    npts = delta_t * 24 * 60 * 20
    
    # 3 sec cadence
    times = Time(np.linspace(t, t + delta_t, int(npts)), format="mjd")

    eph_io = get_body_ephemeris(
        times, body_id="501", step="1m", return_orientation=True
    )
    eph_jup = get_body_ephemeris(
        times, body_id="599", step="1m", return_orientation=False
    )

    eph_list_io.append(eph_io)
    eph_list_jup.append(eph_jup)

In [ ]:
eph_io_stacked = vstack(eph_list_io)
eph_jup_stacked = vstack(eph_list_jupiter)

In [ ]:
# Animate the varying illumination profile of Io
fig, ax = plt.subplots(figsize=(5,5))

map = starry.Map(ydeg=10, reflected=True)

# Plot io
map.show(
    ax=ax,
#     theta=eph_io_stacked["theta"],
    xs=eph_io_stacked["xs"][::50],
    ys=eph_io_stacked["ys"][::50],
    zs=eph_io_stacked["zs"][::50],
)

In [ ]:
def compute_design_matrix_phase(eph_list, reflected=True):
    ydeg = 30
    map = starry.Map(ydeg=ydeg, reflected=reflected)

    A_ = []

    for i in range(len(eph_list)):
        eph = eph_list[i]

        if reflected is True:
            # Get only phase curves in reflected light
            mask = np.all(
                [~eph["ecl_tot"], ~eph["ecl_par"], ~eph["occ_umbra"], ~eph["occ_sun"]],
                axis=0,
            )
        else:
            # Get phase curves in eclipse
            mask = np.all(
                [eph["ecl_tot"], ~eph["occ_umbra"]],
                axis=0,
            )
            
        if mask.sum() > 0.:
            
            # Fix obliquity per light curve, this is fine because
            # obliquity and inclination vary on timescales of years
            obl = np.mean(eph["obl"][mask])
            inc = np.mean(eph["inc"][mask])
            map.obl = obl
            map.inc = inc
            
            theta = np.array(eph["theta"][mask])
            xs = np.array(eph["xs"][mask])
            ys = np.array(eph["ys"][mask])
            zs = np.array(eph["zs"][mask])
           
            # Compute the design matrix for a 1000 points per observing interval         
            if reflected is True:
                m = map.design_matrix(theta=theta[:1000], xs=xs[:1000], ys=ys[:1000], zs=zs[:1000])

            else:
                m = map.design_matrix(theta=theta)

            A_.append(m)

    # Design matrix
    return np.concatenate(A_)


def compute_design_matrix_occ(eph_occulted_list, eph_occultor_list, reflected=False):

    ydeg = 30
    map = starry.Map(ydeg=ydeg, reflected=reflected)

    A_ = []
    xo_list = []
    yo_list = []
    ro_list = []

    for i in range(len(eph_occultor_list)):
        eph_occulted = eph_occulted_list[i]
        eph_occultor = eph_occultor_list[i]

        if reflected is True:
            mask = eph_occulted["occ_sun"]
        else:
            mask = eph_occulted["occ_umbra"]

        # Proceed if mask has at least one True element
        if mask.sum() > 0.0:

            obl = np.mean(eph_occulted["obl"][mask])
            inc = np.mean(eph_occulted["inc"][mask])
            map.obl = obl
            map.inc = inc
            
            theta = np.array(eph_occulted["theta"][mask])
            xs = np.array(eph_occulted["xs"][mask])
            ys = np.array(eph_occulted["ys"][mask])
            zs = np.array(eph_occulted["zs"][mask])

            # Convert everything to units where the radius of Io = 1
            radius_occultor = (
                eph_occultor["ang_width"][mask] / eph_occulted["ang_width"][mask]
            )
            rel_ra = (eph_occultor["RA"][mask] - eph_occulted["RA"][mask]).to(
                u.arcsec
            ) / (0.5 * eph_occulted["ang_width"][mask].to(u.arcsec))
            rel_dec = (eph_occultor["DEC"][mask] - eph_occulted["DEC"][mask]).to(
                u.arcsec
            ) / (0.5 * eph_occulted["ang_width"][mask].to(u.arcsec))

            xo = -rel_ra
            yo = rel_dec
            zo = np.ones(len(yo))
            ro = radius_occultor

            if reflected is True:
                m = map.design_matrix(
                    theta=theta, xs=xs, ys=ys, zs=zs, xo=xo, yo=yo, zo=zo
                )
            else:
                m = map.design_matrix(theta=theta, xo=xo, yo=yo, zo=zo)
                
            A_.append(m)

    return np.concatenate(A_)


def fraction_ns(A, tol):
    """
    Compute the rank of the Fisher information matrix A.TA
    where A is the design matrix.
    """
    ncoeff = len(A[0, :])
    ydeg = int(np.sqrt(ncoeff) - 1)
    vals = np.zeros(ydeg)

    for l in range(ydeg):
        cutoff = int((l + 1) ** 2)

        # Get the rank of the fisher information matrix A.TA
        I = A[:, :cutoff].T.dot(A[:, :cutoff])
        R = np.linalg.matrix_rank(I, tol=tol)

        # Number of coefficients
        C = I.shape[1]

        # Size of null space
        N = C - R

        # Fractional size of null space
        F = N / C
        vals[l] = F

    return vals

In [ ]:
# Phase curve emitted light
A_phase_em = compute_design_matrix_phase(eph_list_io, reflected=False)

# Phase curves in reflected light
A_phase_ref = compute_design_matrix_phase(eph_list_io, reflected=True)

# Occultations in emitted light
A_occ_em = compute_design_matrix_occ(eph_list_io, eph_list_jupiter, reflected=False)

# Occultations in reflected light
A_occ_ref = compute_design_matrix_occ(eph_list_io, eph_list_jupiter, reflected=True)

In [ ]:
# The results are sensitive to the tolerance parameter
# for SVD
tol = 1e-10

ns_emitted = fraction_ns(A_phase_em, tol=tol)
ns_reflected = fraction_ns(A_phase_ref, tol=tol)
ns_occ_emitted = fraction_ns(A_occ_em, tol=tol)
ns_occ_ref = fraction_ns(A_occ_ref, tol=tol)

A_stacked = np.concatenate([A_phase_em, A_phase_ref, A_occ_em, A_occ_ref])
ns_all = fraction_ns(A_stacked, tol=tol)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(np.array(ns_emitted) * 100, "C0.-", label="phase curves emitted light")
ax.plot(np.array(ns_reflected) * 100, "C1.-", label="phase curves reflected light")
ax.plot(np.array(ns_occ_emitted) * 100, "C2.-", label="occultation emitted light")
ax.plot(np.array(ns_occ_ref) * 100, "C3.-", label="occultation reflected light")
ax.plot(np.array(ns_all) * 100, "C4.-", label="combined")

ax.set_xticks(np.arange(0, 35, 5))

ax.grid()
ax.legend(prop={'size': 10})
ax.set_yticks([0, 20, 40, 60, 80, 100])
ax.set_xlabel(r"$l$")
ax.set_ylabel("fraction of nullspace [percent]")

### Mutual occultations of Galilean moons

In [ ]:
import os
# Get ephemeris data from optical light curves from the PHEMU campagin
path = "../data/phemu_catalog/2009/"

eph_list_io = []
eph_list_occ = []

for file in os.listdir(path):
    # Grab date
    y, m, d = file[1:5], file[5:7], file[7:9]
    date_mjd = Time(f"{y}-{m}-{d}", format="isot", scale="utc").to_value("mjd")

    lc = np.genfromtxt(os.path.join(path, file))
    times_mjd = date_mjd + lc[:, 0] / (60 * 24)
    times = Time(times_mjd, format='mjd')
    
    body_id = None
    
    if ("2o1" in file):
        body_id = "502"
    elif ("3o1" in file):
        body_id = "503"
    elif ("4o1" in file):
        body_id = "504"
        
    if body_id is not None:
        eph_io = get_body_ephemeris(
            times, body_id="501", step="1m", return_orientation=True
        )
        eph_occ = get_body_ephemeris(
            times, body_id=body_id, step="1m", return_orientation=True
        )

        eph_list_io.append(eph_io)
        eph_list_occ.append(eph_occ)

In [ ]:
eph_io_stacked = vstack(eph_list_io)
eph_occ_stacked = vstack(eph_list_occ)

In [ ]:
from scipy.linalg import block_diag

obl = eph_io_stacked["obl"]
inc = eph_io_stacked["inc"]

theta = np.array(eph_io_stacked["theta"])

# Convert everything to units where the radius of Io = 1
radius_occultor = (
    eph_occ_stacked["ang_width"] / eph_io_stacked["ang_width"]
)
rel_ra = (eph_occ_stacked["RA"] - eph_io_stacked["RA"]).to(
    u.arcsec
) / (0.5 * eph_io_stacked["ang_width"].to(u.arcsec))
rel_dec = (eph_occ_stacked["DEC"] - eph_io_stacked["DEC"]).to(
    u.arcsec
) / (0.5 * eph_io_stacked["ang_width"].to(u.arcsec))

xo = -rel_ra
yo = rel_dec
zo = np.ones(len(yo))
ro = radius_occultor

# Rotate to coordinate system where the obliquity of Io
theta = - obl.to(u.rad)
c, s = np.cos(theta), np.sin(theta)
R = np.array(([c, -s], [s, c]))
R = np.moveaxis(R, 0, 1)
R_list = list(R.T)
np.shape(R)

In [ ]:
# Rotate all (x, y) pairs at once
R_block = block_diag(*R_list)
res = np.dot(R_block, np.stack([xo, yo]).flatten(order='F'))
res = res.reshape((len(xo), 2))
xo_rot, yo_rot = res[:, 0], res[:, 1]

In [ ]:
from matplotlib.animation import FuncAnimation

fig = plt.figure(figsize=(5,5))
# fig.set_dpi(100)

ax = plt.axes(xlim=(-2, 2), ylim=(-2, 2))
ax.set_aspect('equal')
circ = plt.Circle((0, 0), 1., fc='C1')
ax.add_patch(circ)
circ_occ = plt.Circle((2, -2), 0.75, fc='black')

def init():
    circ_occ.center = (2, 2)
    circ_occ.set_radius(0.1)
    ax.add_patch(circ_occ)
    
    return circ_occ,

def animate(i):
    x, y = xo_rot[i], yo_rot[i]
    circ_occ.center = (x, y)
    circ_occ.set_radius(ro[i])
    
    return circ_occ,

idcs = np.linspace(0, len(xo_rot)- 1, int(len(xo_rot)/10),  dtype=int)

anim = FuncAnimation(fig, animate, 
                               init_func=init, 
                               frames=idcs, 
                               interval=20,
                               blit=True)

In [ ]:
from IPython.display import HTML
HTML(anim.to_html5_video())

In [ ]:
def compute_design_matrix_occ_mutual(eph_occulted_list, eph_occultor_list, reflected=False):
    ydeg = 30
    map = starry.Map(ydeg=ydeg, reflected=reflected)

    A_ = []

    for i in range(len(eph_occultor_list)):
        eph_occulted = eph_occulted_list[i]
        eph_occultor = eph_occultor_list[i]

        
        if reflected is True:
            mask = np.all([~eph_occulted["ecl_tot"], ~eph_occulted["ecl_par"]], axis=0)
        else:
            mask = eph_occulted["ecl_tot"]

        # Proceed if mask has at least one True element
        if mask.sum() > 0.0:
            obl = np.mean(eph_occulted["obl"][mask])
            inc = np.mean(eph_occulted["inc"][mask])
            theta = np.array(eph_occulted["theta"][mask])
            xs = np.array(eph_occulted["xs"][mask])
            ys = np.array(eph_occulted["ys"][mask])
            zs = np.array(eph_occulted["zs"][mask])

            # Convert everything to units where the radius of Io = 1
            radius_occultor = (
                eph_occultor["ang_width"][mask] / eph_occulted["ang_width"][mask]
            )
            rel_ra = (eph_occultor["RA"][mask] - eph_occulted["RA"][mask]).to(
                u.arcsec
            ) / (0.5 * eph_occulted["ang_width"][mask].to(u.arcsec))
            rel_dec = (eph_occultor["DEC"][mask] - eph_occulted["DEC"][mask]).to(
                u.arcsec
            ) / (0.5 * eph_occulted["ang_width"][mask].to(u.arcsec))

            xo = -rel_ra
            yo = rel_dec
            zo = np.ones(len(yo))
            ro = radius_occultor

            map.obl = obl
            map.inc = inc

            if reflected is True:
                m = map.design_matrix(
                    theta=theta, xs=xs, ys=ys, zs=zs, xo=xo, yo=yo, zo=zo
                )
            else:
                m = map.design_matrix(theta=theta, xo=xo, yo=yo, zo=zo)
                
            A_.append(m)

    # Design matrix
    return np.concatenate(A_)

In [ ]:
# Occultations in reflected light
A_occ_mutual_em = compute_design_matrix_occ_mutual(eph_list_io, eph_list_occ, reflected=False)
A_occ_mutual_ref = compute_design_matrix_occ_mutual(eph_list_io, eph_list_occ, reflected=True)

ns_mutual_em = fraction_ns(A_occ_mutual_em, tol=1e-10)
ns_mutual_ref = fraction_ns(A_occ_mutual_ref, tol=1e-10)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(np.array(ns_mutual_em) * 100, "C0.-", label="mutual occultations emitted light")
ax.plot(np.array(ns_mutual_ref) * 100, "C1.-", label="mutual occultations reflected light")

ax.set_xticks(np.arange(0, 35, 5))

ax.grid()
ax.legend(prop={'size': 10})
# ax.set_xticks(l)
ax.set_yticks([0, 20, 40, 60, 80, 100])
ax.set_xlabel(r"$l$")
ax.set_ylabel("fraction of nullspace [percent]")